In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from importlib import reload 

In [3]:
from deeprank.dataset import DataLoader, PairGenerator, ListGenerator
from deeprank import utils

In [4]:
seed = 1234
torch.manual_seed(seed)

In [5]:
loader = DataLoader('./config/letor07_mp_fold1.model')

[./data/letor/r5w/word_dict.txt]
	Word dict size: 193367
[./data/letor/r5w/qid_query.txt]
	Data size: 1692
[./data/letor/r5w/docid_doc.txt]
	Data size: 65323
[./data/letor/r5w/embed_wiki-pdc_d50_norm]
	Embedding size: 109282
Generate numpy embed: (193368, 50)


In [6]:
import json
letor_config = json.loads(open('./config/letor07_mp_fold1.model').read())
#device = torch.device("cuda")
device = torch.device("cpu")

In [7]:
Letor07Path = letor_config['data_dir']

letor_config['fill_word'] = loader._PAD_
letor_config['embedding'] = loader.embedding
letor_config['feat_size'] = loader.feat_size
letor_config['vocab_size'] = loader.embedding.shape[0]
letor_config['embed_dim'] = loader.embedding.shape[1]
letor_config['pad_value'] = loader._PAD_

pair_gen = PairGenerator(rel_file=Letor07Path + '/relation.train.fold%d.txt'%(letor_config['fold']), 
                         config=letor_config)

[./data/letor/r5w/relation.train.fold1.txt]
	Instance size: 47828
Pair Instance Count: 325439


In [8]:
from deeprank import select_module
from deeprank import rank_module

In [9]:
select_net = select_module.IdentityNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)

In [10]:
letor_config['q_limit'] = 20
letor_config['d_limit'] = 500
letor_config['max_match'] = 5
letor_config['win_size'] = 5
select_net = select_module.QueryCentricNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)

In [11]:
letor_config['q_limit'] = 20
letor_config['d_limit'] = 500
letor_config['max_match'] = 5
letor_config['win_size'] = 5
letor_config['finetune_embed'] = False
select_net = select_module.PointerNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)

In [12]:
# letor_config['simmat_channel'] = 1
# letor_config['conv_params'] = [(8, 2, 10)]
# letor_config['fc_params'] = [50]
# letor_config['dpool_size'] = [3, 10]
# letor_config['lr'] = 0.001
# letor_config['finetune_embed'] = False
# rank_net = rank_module.MatchPyramidNet(config=letor_config)
# rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
# rank_net.train()
# optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [13]:
letor_config['simmat_channel'] = 1
letor_config['conv_params'] = [(8, 3, 3)]
letor_config['fc_params'] = [200]
letor_config['dpool_size'] = [3, 10]
letor_config['lr'] = 0.001
letor_config['finetune_embed'] = False
rank_net = rank_module.MatchPyramidNet(config=letor_config)
rank_net = rank_net.to(device)
rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
rank_net.train()
optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [14]:
def to_device(*variables):
    return (torch.from_numpy(variable).to(device) for variable in variables)

In [15]:
import time
start_t = time.time()
for i in range(150):
    X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F = \
        pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
    X1, X1_len, X2, X2_len, Y, F = \
        to_device(X1, X1_len, X2, X2_len, Y, F)
    X1, X2, X1_len, X2_len = select_net(X1, X2, X1_len, X2_len, X1_id, X2_id)
    X2, X2_len = utils.data_adaptor(X2, X2_len, select_net, rank_net, letor_config)
    output = rank_net(X1, X2, X1_len, X2_len, 0)
    loss = rank_net.pair_loss(output, Y)
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
end_t = time.time()
print('Time Cost: %s s' % (end_t-start_t))

0.9980525374412537
0.985288679599762
0.9936937689781189
0.9841791391372681
0.9776427149772644
0.9458248019218445
0.9543267488479614
0.9224098324775696
0.9914007186889648
1.0139590501785278
0.9459705948829651
0.9567182064056396
0.8536685705184937
1.0058071613311768
0.8879978656768799
0.8629388809204102
0.8691524863243103
0.9593749046325684
0.9348424077033997
0.909866988658905
0.7485393285751343
0.9117458462715149
0.9740102291107178
0.8736965656280518
0.8772196769714355
0.9152899384498596
1.0229172706604004
0.9273777604103088
0.8982543349266052
0.8502958416938782
0.8788208365440369
0.8815294504165649
0.9403094053268433
0.859752893447876
0.891225278377533
0.8960438966751099
0.8490146398544312
0.8890631794929504
0.8776953220367432
0.8999330997467041
0.8951934576034546
0.8349348902702332
0.9231674671173096
0.931037425994873
0.7456488013267517
0.9343036413192749
0.8365973830223083
0.9466518759727478
0.8679417371749878
0.8805765509605408
0.9309817552566528
0.8150728940963745
0.809029877185821

In [16]:
torch.save(select_net.state_dict(), "identity.ckpt")
torch.save(rank_net.state_dict(), "matchpyramid.ckpt")

In [17]:
torch.save(select_net, "identity.model")
torch.save(rank_net, "matchpyramid.model")

In [18]:
rank_net

MatchPyramidNet(
  (embedding): Embedding(193368, 50, padding_idx=193367)
  (conv_sequential): Sequential(
    (0): Conv2d(1, 8, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  )
  (dpool_layer): AdaptiveMaxPool2d(output_size=[3, 10])
  (fc_sequential): Sequential(
    (0): Linear(in_features=240, out_features=200, bias=True)
  )
  (out_layer): Linear(in_features=200, out_features=1, bias=True)
)

In [19]:
select_net_e = torch.load(f='identity.model')
rank_net_e = torch.load(f='matchpyramid.model')

list_gen = ListGenerator(rel_file=Letor07Path+'/relation.test.fold%d.txt'%(letor_config['fold']),
                         config=letor_config)
map_v = 0.0
map_c = 0.0

with torch.no_grad():
    for X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F in \
        list_gen.get_batch(data1=loader.query_data, data2=loader.doc_data):
        #print(X1.shape, X2.shape, Y.shape)
        X1, X1_len, X2, X2_len, Y, F = to_device(X1, X1_len, X2, X2_len, Y, F)
        X1, X2, X1_len, X2_len = select_net_e(X1, X2, X1_len, X2_len, X1_id, X2_id)
        X2, X2_len = utils.data_adaptor(X2, X2_len, select_net, rank_net, letor_config)
        #print(X1.shape, X2.shape, Y.shape)
        pred = rank_net_e(X1, X2, X1_len, X2_len, 0)
        map_o = utils.eval_MAP(pred.tolist(), Y.tolist())
        #print(pred.shape, Y.shape)
        map_v += map_o
        map_c += 1.0
    map_v /= map_c

print('[Test]', map_v)

[./data/letor/r5w/relation.test.fold1.txt]
	Instance size: 13652
List Instance Count: 336
[Test] 0.37934977287443
